# Running the PlanaR Pipeline on an Image
---
## **Description:**
This notebook provides users with a tutorial-esque approach to understand the preliminary work that went into the PlanaR pipeline (not to be confused with the OPlanaR pipeline discussed in the report). This pipeline was used to measure properties of MEndR muscle fibers. It uses classical computer vision techniques (connectvity, contour sketching, distance transforms, etc.) and a few shallow machine learning techniques (DBSCAN, Random Forest, etc.) to measure the properties of fibers. The fundamental concept is to take planar cross sections along the image length. This idea is not as good as orthogonal projections, however.

Although the methods below are not state-of-the-art (deep learning) or the best solution for the problem (compared to the OPlanaR described in the project report), the motivation behind this notebook is three-fold:
1. To provide useful functions for users that can act as a starting point for their image analysis tasks
2. To extend the concepts behind this model to other tasks. Some extensions, off the top of my head, include:
    - Identifying unique cell types based on morphological features (eg: cancer cell types that look different to other cell types)
    - Localizing cells (eg: live cell imaging) which are disjoint
    - Labelling cells to create datasets that can be used to train more advanced deep learning models 
3. To help researchers get acquainted with image analysis packages in Python (if they decide to migrate from MATLAB or R)

The idea of interactive and open-source notebooks was in part inspired by the [Allen Institute](https://www.allencell.org/segmenter.html). 

If any difficulties or issues arise, please reach out to me at [dylan.mendonca@mail.utoronto.ca](mailto:dylan.mendonca@mail.utoronto.ca) and we will do our best to help.<br>

<hr>

### Prerequisites:
This notebook assumes that:
1. You are familiar with the functions created for this project and described in the tutorial notebook [here](tutorial.ipynb)
2. You have read the MEng project report

<hr>

**Authors:** Dylan Mendonca<br>
**Date of Last Revision:** October-8-2020<br>
<hr>

## Importing Packages

In [1]:
import sys
import numpy as np
from aicsimageio import AICSImage
from itkwidgets import view
from skimage import morphology

sys.path.append("../")
from utilities.format_helpers import array_to_image, image_to_array
from utilities.visualize import side_by_side_3dview
from utilities.sliceBySliceSegmenter import SliceBySliceSegmenter

### Step 1: Import a Graph Cuts-based Segmentation
The PlanaR pipeline makes use of a Graph Cuts-based thresholding strategy mentioned in the project report. 

The MATLAB code for doing the graph cuts thresholding can be found [here](../../matlab/Graphcut_Fibers_16.m). The script to run **only** the segmentation has been included [here](../../matlab/RunningScript_Thresholding.m) while the script to run **both** thresholding and bulk diameter measurements can be found [here](../../matlab/RunningScript_ThresholdingAndBulkMeasurementMethod.m). 

For more information on the aformentioned MATLAB code, please contact [Amir Meysami](mailto:amir.meysamifard@mail.utoronto.ca).

In [2]:
# SPACING
SPACING = [0.497, 0.497, 3]

# Image directory
image_dir = '../../../data/GFP-seg.tiff'

# Loading the image
reader = AICSImage(image_dir) 
IMG = reader.data.astype(np.bool)
segmented = IMG[0,0,:,:,:]

In [3]:
view(array_to_image(segmented, spacing=SPACING))

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

### Step 2: Remove Small Objects
There are a view small objects, so we need to remove them

In [4]:
##############
# PARAMETERS #
MIN_OBJECT_SIZE = 1000
CONNECTIVITY = 26
##############

small_objs_removed = morphology.remove_small_objects(segmented, min_size=MIN_OBJECT_SIZE, connectivity=CONNECTIVITY)

In [5]:
# Let's see the result
side_by_side_3dview(small_objs_removed, segmented, spacing=SPACING)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

### Step 3: Planar Cuts & Measurements Along a Dimension
This section of the pipeline will iterate through slices in a specified plane (either XZ (dim=1) or YZ (dim=2)), identify unique disconnected blobs, label them, and measure many properties about them (area, perimeter, convex area, etc.). This functionality is carried out by the SliceBySliceSegmenter object, whose code is located [here](../utilities/sliceBySliceSegmenter.py).

The most important attributes of the object are:
- `segmenter.zoomed_object`: The new object after accounting for voxel spacing. This is the object that the segmenter works with
- `segmenter.labelled_object`: The object where each planes' blobs have been labelled. It's important to note that these labels might not necessarily coincide with each other between planes i.e. label 1 in plane 0 and label 1 in plane 1 might not belong to the same 3D fiber. It is also important to note that this object is the same size as zoomed object, so that voxel spacing is 1x1x1
- `segmenter.slice_info`: A DataFrame containing all the slice numbers, labels, and measured properties of each label

In [6]:
# The segmenter object does all the measurements within this line of code
segmenter = SliceBySliceSegmenter(small_objs_removed, 'xz', SPACING, 'Connectivity')

In [7]:
# Here's another cool little feature to make sure that things are okay
segmenter.widget()

In [8]:
# Here's the dataframe of information
segmenter.slice_info.head()

,area,bbox-0,bbox-1,bbox-2,bbox-3,bbox_area,centroid-0,centroid-1,convex_area,convex_image,...,label,major_axis_length,minor_axis_length,no_of_objects,perimeter,slice,solidity,convexity_per,convexity_area,roundness
0,62,79,200,89,211,110,83.419355,205.467742,89,"[[True, True, True, True, True, False, False, ...",...,1,13.389653,9.169534,4,45.656854,0,0.696629,0.755314,1.435484,0.655140
1,3,80,123,83,124,3,81.000000,123.000000,3,"[[True], [True], [True]]",...,2,3.265986,0.000000,4,1.000000,0,1.000000,1.000000,1.000000,37.699112
2,15,83,239,89,242,18,85.466667,240.066667,16,"[[False, True, True], [True, True, True], [Tru...",...,3,6.510742,3.085668,4,12.242641,0,0.937500,1.047848,1.066667,1.145392
3,34,86,286,92,293,42,88.764706,288.735294,38,"[[True, True, True, True, True, False, False],...",...,4,7.835405,6.146216,4,22.242641,0,0.894737,0.936419,1.117647,0.984863
4,9,80,121,83,124,9,81.000000,122.000000,9,"[[True, True, True], [True, True, True], [True...",...,1,3.265986,3.265986,5,8.000000,1,1.000000,1.000000,1.000000,1.767146
